In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
train_df = pd.read_csv("Datasets/train.csv")

In [39]:
train_df[train_df["Embarked"].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [40]:
# Feature Engineering

clean_df = pd.concat([train_df, pd.get_dummies(train_df.Pclass, prefix="Class")], axis=1)
clean_df = pd.concat([clean_df, pd.get_dummies(train_df.Sex)], axis=1)
clean_df = pd.concat([clean_df, pd.get_dummies(train_df.Embarked, dummy_na=True, prefix="Embarked")], axis=1)
clean_df.drop(["Ticket", "Name", "PassengerId", "Pclass", "Sex", "Embarked", "Cabin"], axis=1, inplace=True)
clean_df

,Survived,Age,SibSp,Parch,Fare,Class_1,Class_2,Class_3,female,male,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,0,22.0,1,0,7.2500,0,0,1,0,1,0,0,1,0
1,1,38.0,1,0,71.2833,1,0,0,1,0,1,0,0,0
2,1,26.0,0,0,7.9250,0,0,1,1,0,0,0,1,0
3,1,35.0,1,0,53.1000,1,0,0,1,0,0,0,1,0
4,0,35.0,0,0,8.0500,0,0,1,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,27.0,0,0,13.0000,0,1,0,0,1,0,0,1,0
887,1,19.0,0,0,30.0000,1,0,0,1,0,0,0,1,0
888,0,NaN,1,2,23.4500,0,0,1,1,0,0,0,1,0
889,1,26.0,0,0,30.0000,1,0,0,0,1,1,0,0,0


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Class_1,Class_2,Class_3
0,0,3,male,22.0,1,0,7.2500,NaN,S,0,0,1
1,1,1,female,38.0,1,0,71.2833,C85,C,1,0,0
2,1,3,female,26.0,0,0,7.9250,NaN,S,0,0,1
3,1,1,female,35.0,1,0,53.1000,C123,S,1,0,0
4,0,3,male,35.0,0,0,8.0500,NaN,S,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,NaN,S,0,1,0
887,1,1,female,19.0,0,0,30.0000,B42,S,1,0,0
888,0,3,female,NaN,1,2,23.4500,NaN,S,0,0,1
889,1,1,male,26.0,0,0,30.0000,C148,C,1,0,0
